In [61]:
import random
from datetime import datetime, timedelta
import pandas as pd # hoping to not need
import numpy as np

class BedModel:
    """
    Generalised Inpatient Bed Model
    
    Example:
    
    warmup_n = 574

    source_prob = {"Emergency Department": 0.8, "Non-ED Admissions": 0.14, "Waiting List": 0.06}

    category_prob = {"Emergency Department": {'Elective':0, 'Surgical Emergency':0.2, 'Medical Emergency':0.8},
                 "Non-ED Admission": {'Elective':0, 'Surgical Emergency':0.4, 'Medical Emergency':0.6}
                 }

    los_distributions = {
        'Emergency Department': {'Elective': (1, 0.5), 'Surgical Emergency': (2, 0.7), 'Medical Emergency': (3, 1)},
        'Non-ED Admission': {'Elective': (1.5, 0.6), 'Surgical Emergency': (2.5, 0.8), 'Medical Emergency': (3.5, 1.2)},
        'Elective': {'Elective': (2, 0.7)}
    }

    hospital = BedModel(n_elective_beds=40,
                        n_surgical_emergency_beds=150,
                        n_medical_emergency_beds=450,
                        n_escalation_beds=20, 
                        source_probability=source_prob, 
                        category_probability=category_prob, 
                        los_distributions=los_distributions)
    
    hospital.warm_up_model(warmup_number=warmup_n)
    
    """
    def __init__(self, n_elective_beds, n_surgical_emergency_beds, n_medical_emergency_beds, n_escalation_beds, source_probability, category_probability, los_distributions, time_matrix):
        """
        :param num_elective_beds:
        :param num_surgical_emergency_beds:
        :param num_medical_emergency_beds:
        :param num_escalation_beds:
        """
        # Total Beds
        self.n_elective_beds = n_elective_beds # ? can these four use a dictionary to reduce n parameters
        self.n_surgical_emergency_beds = n_surgical_emergency_beds
        self.n_medical_emergency_beds = n_medical_emergency_beds
        self.n_escalation_beds = n_escalation_beds
        
        # Global Variables
        self.los_distributions = los_distributions
        self.category_probability = category_probability
        self.source_probability = source_probability
        self.time_matrix = time_matrix
        
        # Beds occupied
        self.occupied_elective_beds = []
        self.occupied_surgical_emergency_beds = []
        self.occupied_medical_emergency_beds = []
        self.occupied_escalation_beds = []
        
        # All Patients
        self.patient_master = [] # A master holding place for the warm up patients so it can be replicated each run
        
        # Holding Places
        self.ed_queue = [] # If an emergency patient they wait here (trolley wait)
        self.non_ed_queue = [] # If an emergency patient they wait here
        self.Elective_queue = [] # If an Elective patient they wait here
        self.Elective_cancellations = [] # If a patient from the Waiting List cannot be admitted they are cancelled

        # Metrics
        self.record_available_beds = {'Elective': [], 'surgical emergency': [], 'medical emergency': [], 'escalation': []}
        self.record_n_occupied_beds = {'Elective': [], 'surgical emergency': [], 'medical emergency': [], 'escalation': []}
        self.record_n_outliers = {'Elective': [], 'surgical emergency': [], 'medical emergency': []}
        self.record_n_escalation = []
        self.record_n_admissions_by_hour = []
        self.record_n_discharges_by_hour = []
        self.record_mean_length_of_stay = [] # wondering if there is any use in recording this
        self.record_mean_ed_queue = []
        self.record_mean_non_ed_queue = []
        self.record_n_trolley_waits = []
        self.record_n_cancellations = []


    # Tools
    
    # These functions are used as tools throughout the model

    def unique_id_generator(self):
        """
        Prevents Patient IDS being duplicated in self.patient when generating a new patient
        
        ?? Change to yield id  
        ?? double check if this is checking the UID in the correct position
        :return: 
        """
        
        all_patients = self.patient_master + self.occupied_escalation_beds + self.occupied_elective_beds + self.occupied_medical_emergency_beds + self.occupied_surgical_emergency_beds # eliminate the need for a master patient index to always exist
        
        while True:
            new_id = random.randint(100000, 999999) # IDs based on a 6-digit number 
            if new_id not in [patient[0] for patient in all_patients]:
                return new_id


    def patient_generator(self, n, warm=False, source_=None):
        """
        Create stochastic patients based on probability and distributions
        :param source_: the type of patient to generate
        :param warm: whether to generate from warmup
        :param n: Number of patients to generate
        :return: patients to patient_master
        """
        if warm:
            for i in range(0, n):
        
                patient_id, source, category, los = None, None, None, None
        
                patient_id = self.unique_id_generator()
        
                source = np.random.choice(['Emergency Department', 'Non-ED Admission', 'Elective'],
                                          1,
                                          p=[self.source_probability.get('Emergency Department'),
                                             self.source_probability.get('Non-ED Admissions'),
                                             self.source_probability.get('Waiting List')]).item(0)
        
                # Get the corresponding probability that the Patient will be Elective, Surgical Emergency or Medical Emergency 
        
                if source == 'Emergency Department':
                    category = np.random.choice(['Elective', 'Surgical Emergency', 'Medical Emergency'],
                                                1,
                                                p=[self.category_probability.get(source)['Elective'],
                                                   self.category_probability.get(source)['Surgical Emergency'],
                                                   self.category_probability.get(source)['Medical Emergency']]).item(0)
        
                    # Pick from a los distribution using the mean and sigma !! Assuming that LOS is log normal just while building, need to check this
        
                    if category == 'Elective':
                        los = int(np.random.lognormal(self.los_distributions.get(source)['Elective'][0],
                                                      self.los_distributions.get(source)['Elective'][1],
                                                      1).item(0))
        
                    elif category == 'Surgical Emergency':
                        los = int(np.random.lognormal(self.los_distributions.get(source)['Surgical Emergency'][0],
                                                      self.los_distributions.get(source)['Surgical Emergency'][1],
                                                      1).item(0))
        
                    elif category == 'Medical Emergency':
                        los = int(np.random.lognormal(self.los_distributions.get(source)['Medical Emergency'][0],
                                                      self.los_distributions.get(source)['Medical Emergency'][1],
                                                      1).item(0))
        
                elif source == 'Non-ED Admission':
                    category = np.random.choice(['Elective', 'Surgical Emergency', 'Medical Emergency'],
                                                1,
                                                p=[self.category_probability.get(source)['Elective'],
                                                   self.category_probability.get(source)['Surgical Emergency'],
                                                   self.category_probability.get(source)['Medical Emergency']]).item(0)
        
                    if category == 'Elective':
                        los = int(np.random.lognormal(self.los_distributions.get(source)['Elective'][0],
                                                      self.los_distributions.get(source)['Elective'][1],
                                                      1).item(0))
        
                    elif category == 'Surgical Emergency':
                        los = int(np.random.lognormal(self.los_distributions.get(source)['Surgical Emergency'][0],
                                                      self.los_distributions.get(source)['Surgical Emergency'][1],
                                                      1).item(0))
        
                    elif category == 'Medical Emergency':
                        los = int(np.random.lognormal(self.los_distributions.get(source)['Medical Emergency'][0],
                                                      self.los_distributions.get(source)['Medical Emergency'][1],
                                                      1).item(0))
        
                elif source == 'Elective':
                    # Waiting List patients will always be admitted as a Elective
                    category = 'Elective'
                    los = int(np.random.lognormal(self.los_distributions.get(source)['Elective'][0],
                                                  self.los_distributions.get(source)['Elective'][1],
                                                  1).item(0))
        
                patient_data = [patient_id, source, category, los]
        
                self.patient_master.append(patient_data)
                
        else:
            if source_:

                for i in range(0, n):
                
                    patient_id, source, category, los = None, None, None, None
                    
                    if source_=='Emergency Department':
                        
                        patient_id = self.unique_id_generator()
                        
                        source = 'Emergency Department'
    
                        category = np.random.choice(['Elective', 'Surgical Emergency', 'Medical Emergency'],
                                                    1,
                                                    p=[self.category_probability.get(source)['Elective'], # Don't think this needs to be in, 
                                                       self.category_probability.get(source)['Surgical Emergency'],
                                                       self.category_probability.get(source)['Medical Emergency']]).item(0)
    
                        if category == 'Elective':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Elective'][0],
                                                          self.los_distributions.get(source)['Elective'][1],
                                                          1).item(0))
    
                        elif category == 'Surgical Emergency':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Surgical Emergency'][0],
                                                          self.los_distributions.get(source)['Surgical Emergency'][1],
                                                          1).item(0))
    
                        elif category == 'Medical Emergency':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Medical Emergency'][0],
                                                          self.los_distributions.get(source)['Medical Emergency'][1],
                                                          1).item(0))
    
                    elif source_=='Non-ED Admissions':
    
                        patient_id = self.unique_id_generator()
    
                        source = 'Non-ED Admissions'
    
                        category = np.random.choice(['Elective', 'Surgical Emergency', 'Medical Emergency'],
                                                    1,
                                                    p=[self.category_probability.get(source)['Elective'],
                                                       self.category_probability.get(source)['Surgical Emergency'],
                                                       self.category_probability.get(source)['Medical Emergency']]).item(0)
    
                        if category == 'Elective':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Elective'][0],
                                                          self.los_distributions.get(source)['Elective'][1],
                                                          1).item(0))
    
                        elif category == 'Surgical Emergency':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Surgical Emergency'][0],
                                                          self.los_distributions.get(source)['Surgical Emergency'][1],
                                                          1).item(0))
    
                        elif category == 'Medical Emergency':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Medical Emergency'][0],
                                                          self.los_distributions.get(source)['Medical Emergency'][1],
                                                          1).item(0))
                        
                    elif source_=='Waiting List':
    
                        patient_id = self.unique_id_generator()
    
                        source = 'Waiting List'
    
                        category = np.random.choice(['Elective', 'Surgical Emergency', 'Medical Emergency'],
                                                    1,
                                                    p=[self.category_probability.get(source)['Elective'],
                                                       self.category_probability.get(source)['Surgical Emergency'],
                                                       self.category_probability.get(source)['Medical Emergency']]).item(0)
    
                        if category == 'Elective':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Elective'][0],
                                                          self.los_distributions.get(source)['Elective'][1],
                                                          1).item(0))
    
                        elif category == 'Surgical Emergency':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Surgical Emergency'][0],
                                                          self.los_distributions.get(source)['Surgical Emergency'][1],
                                                          1).item(0))
    
                        elif category == 'Medical Emergency':
                            los = int(np.random.lognormal(self.los_distributions.get(source)['Medical Emergency'][0],
                                                          self.los_distributions.get(source)['Medical Emergency'][1],
                                                          1).item(0))
                        
                    patient_data = [patient_id, source, category, los]
    
                    return patient_data
            
            else:
                raise Exception('A source has not been specified for the patient to be generated')
            
            
            
    # This function acts as a warm-up, setting the starting figures in the simulation, so it does not begin with an empty system
            
    def warm_up_model(self, warmup_number):
        """
        
        :param warmup_number: Number of patients to be generated at warm up
        :return: 
        """
        
        if warmup_number > self.n_surgical_emergency_beds + self.n_elective_beds + self.n_medical_emergency_beds + self.n_escalation_beds:
            raise ValueError("The number of patients at warm-up cannot exceed the beds available")
        
        else:
            self.patient_generator(n=warmup_number, warm=True)
        
        # Admit patients              
        self.admit_patient(warm=True)
    
                

    # Calculation Functions
    
    # These functions are to calculate the various metrics in the simulation
                
    def calculate_outliers(self):
        """
        Add the number of outliers to  self.record_n_outliers existing in the Elective, Medical Emergency and Surgical Emergency Beds
        !! Escalation usage is recorded in a different function 
        """

        Elective_outliers = 0
        medical_outliers = 0
        surgical_outliers = 0

        for outlier in self.occupied_medical_emergency_beds:
            if outlier[2] == 'Elective':
                Elective_outliers += 1
            elif outlier[2] == 'Surgical Emergency':
                surgical_outliers += 1
        
        for outlier in self.occupied_surgical_emergency_beds:
            if outlier[2] == 'Elective':
                Elective_outliers += 1
            elif outlier[2] == 'Medical Emergency':
                medical_outliers += 1

        for outlier in self.occupied_elective_beds:
            if outlier[2] == 'Medical Emergency':
                medical_outliers += 1
            elif outlier[2] == 'Surgical Emergency':
                surgical_outliers += 1
        
        self.record_n_outliers['Elective'].append(Elective_outliers)
        self.record_n_outliers['surgical emergency'].append(surgical_outliers)
        self.record_n_outliers['medical emergency'].append(medical_outliers)
        
    def calculate_escalation(self):
        """
        
        :return: 
        """
        escalation = 0 
        
        if self.occupied_escalation_beds:
            self.record_n_escalation.append(len(self.occupied_escalation_beds))
        else: 
            self.occupied_escalation_beds.append(escalation)
            
                  
    def calculate_available_beds(self):
        """
        Using the bed base calculate the remaining by subtracting the number in the corresponding occupied beds
        :return: 
        """
        remaining_medical_emergency = self.n_medical_emergency_beds - self.occupied_medical_emergency_beds[-1]
        remaining_surgical_emergency = self.n_surgical_emergency_beds - self.occupied_surgical_emergency_beds[-1]
        remaining_elective = self.n_elective_beds - self.occupied_elective_beds[-1]
        remaining_escalation = self.n_escalation_beds - self.occupied_escalation_beds[-1]
        
        self.record_available_beds['medical emergency'].append(remaining_medical_emergency)
        self.record_available_beds['surgical emergency'].append(remaining_surgical_emergency)
        self.record_available_beds['Elective'].append(remaining_elective)
        self.record_available_beds['escalation'].append(remaining_escalation)
           
    # Core Functions
    
    # All these functions are used in the running of the simulation
          
    def discharge_patient(self):
        """
        This function should store the number of patients ready for discharge and then remove them from the system
        :return: number of discharges to self.record_n_discharges_by_hour
        """
        
        discharged = 0
        
        if len(self.occupied_medical_emergency_beds) > 0:

            discharged += [sublist for sublist in self.occupied_medical_emergency_beds if sublist[3] == 0]
            self.occupied_medical_emergency_beds = [sublist for sublist in t if sublist[3] != 0]
        
        if len(self.occupied_surgical_emergency_beds) > 0:
            discharged += [sublist for sublist in self.occupied_surgical_emergency_beds if sublist[3] == 0]
            self.occupied_surgical_emergency_beds = [sublist for sublist in t if sublist[3] != 0]
        
        if len(self.occupied_elective_beds) > 0:
            discharged += [sublist for sublist in self.occupied_elective_beds if sublist[3] == 0]
            self.occupied_elective_beds = [sublist for sublist in t if sublist[3] != 0]
        
        if len(self.occupied_escalation_beds) > 0:
            discharged += [sublist for sublist in self.occupied_escalation_beds if sublist[3] == 0]
            self.occupied_escalation_beds = [sublist for sublist in t if sublist[3] != 0]
        
        self.record_n_discharges_by_hour.append(discharged)
        
    
    def cancel_patient(self):
        """
        This function should add any patients left in the Elective queue but where there are no beds
        :param patient_being_cancelled: a list or a number of patients that are being cancelled
        :return: 
        """
          
        self.Elective_cancellations.append(len(self.Elective_queue))
        # Clear all remaining Electives in the queue 
        self.Elective_queue.clear()
        
       
    def admit_patient(self, warm):
        """
        This function should admit patients from the holding areas
        :return: 
        """
        
        if warm:
            for patient in self.patient_master:
                # Prioritise the placement of emergency patients

                if patient[2] == 'Medical Emergency' and len(self.occupied_medical_emergency_beds) < self.n_medical_emergency_beds:
                    self.occupied_medical_emergency_beds.append(patient)

                # If this is not possible place the patients in surgical beds
                elif patient[2] == 'Medical Emergency' and len(self.occupied_medical_emergency_beds) >= self.n_medical_emergency_beds and len(self.occupied_surgical_emergency_beds) < self.n_surgical_emergency_beds:
                    self.occupied_surgical_emergency_beds.append(patient)

                elif patient[2] == 'Surgical Emergency' and len(self.occupied_surgical_emergency_beds) < self.n_surgical_emergency_beds:
                    self.occupied_surgical_emergency_beds.append(patient)

                elif patient[2] == 'Surgical Emergency' and len(self.occupied_surgical_emergency_beds) >= self.n_surgical_emergency_beds and len(self.occupied_medical_emergency_beds) < self.n_medical_emergency_beds:
                    self.occupied_medical_emergency_beds.append(patient)

                elif patient[2] == 'Elective' and len(self.occupied_elective_beds) < self.n_elective_beds:
                    self.occupied_elective_beds.append(patient)

                elif patient[2] == 'Elective' and len(self.occupied_elective_beds) >= self.n_elective_beds and len(self.occupied_surgical_emergency_beds) < self.n_surgical_emergency_beds:
                    self.occupied_surgical_emergency_beds.append(patient)

                elif patient[2] == 'Elective' and len(self.occupied_elective_beds) >= self.n_elective_beds and len(self.occupied_medical_emergency_beds) < self.n_medical_emergency_beds:
                    self.occupied_medical_emergency_beds.append(patient)

                # If patient cannot fit within the core bed base then use escalation beds
                else:
                    self.occupied_escalation_beds.append(patient)
        
    def arrivals(self, hour, weekday):
        """
        This function handles generating the new arrivals each hour and puts them in the holding area ready for the admit function
        :return: 
        """ 
        
        number_being_admitted_emergency_department = self.time_matrix.get('Emergency Department')[weekday][hour]
        number_being_admitted_non_emergency_department = self.time_matrix.get('Non-ED Admission')[weekday][hour]
        number_being_admitted_n_elective = self.time_matrix.get('Elective')[weekday][hour]
        
        if number_being_admitted_emergency_department:
        
            new = self.patient_generator(n=number_being_admitted_emergency_department, source_='Emergency Department')
            
            self.ed_queue.append(new)
        
        if number_being_admitted_non_emergency_department:
            
            new = self.patient_generator(n=number_being_admitted_emergency_department, source_='Elective')

            self.ed_queue.append(new)

        if number_being_admitted_n_elective:

            new = self.patient_generator(n=number_being_admitted_emergency_department, source_='Elective')

            self.ed_queue.append(new)

    
    # End Results
    
    # These functions are used to record the end results of the model and graphically show them
        
    def collect_results(self):
        """
        This function should collect the results into a tabular format
        
        :return: 
        """
    
    def graph_results(self):
        """
        This function should graphically show the results of the simulation
        :return: 
        """

    # Main Function 
    
    # This is the core function called to run the simulation (after set up and warm up)

    def simulate_inpatient_system(self, start_time, end_time, runs=100):
        """
        
        :param start_time: datetime for when the simulation should start
        :param end_time: datetime for when the simulation should end
        :param runs: the number of runs that should be executed to collect results, default: 100 runs
        :return: 
        """
        
        for i in range(runs):
            current_time = start_time
            
            while current_time <= end_time:
                # Beds occupied
                #self.occupiedn_elective_beds = []
                #self.occupied_surgical_emergency_beds = []
                #self.occupied_medical_emergency_beds = []
                #self.occupied_escalation_beds = []
        
                # All Patients
                #self.patient_master = [] # A master holding place for newly generated patients until they are assigned to the correct holder or bed
        
                #Queues / cancellations
                #self.ed_queue = [] # If an emergency patient from ed cannot be admitted they wait here (trolley wait)
                #self.non_ed_queue = [] # If an emergency patient from a non ed source cannot be admitted they wait here
                #self.Elective_cancellations = [] # If a patient from the Waiting List cannot be admitted they are cancelled
        
                # Metrics
                #self.record_available_beds = {'Elective': [], 'surgical emergency': [], 'medical emergency': [], 'escalation': []}
                #self.record_n_occupied_beds = {'Elective': [], 'surgical emergency': [], 'medical emergency': [], 'escalation': []}
                #self.record_n_outliers = {'Elective': [], 'surgical emergency': [], 'medical emergency': []}
                #self.record_n_admissions_by_hour = []
                #self.record_n_discharges_by_hour = []
                #self.record_mean_length_of_stay = []
                #self.record_mean_ed_queue = []
                #self.record_mean_non_ed_queue = []
                
                #Finished
                self.record_n_escalation.append(len(self.occupied_escalation_beds))
                self.record_n_trolley_waits.append(sum(len(self.non_ed_queue), len(self.ed_queue)))
                
                # Count how many Elective patients couldn't be admitted and were moved to the cancelled parameter
                self.record_n_cancellations.append(len(self.cancellations))
                # Clear the Cancellations after they have been recorded
                self.cancellations.clear()
                
                current_time += timedelta(hours=1)
        



In [62]:


warmup_n = 574

source_prob = {"Emergency Department": 0.8, "Non-ED Admissions": 0.14, "Waiting List": 0.06}

category_prob = {"Emergency Department": {'Elective':0, 'Surgical Emergency':0.2, 'Medical Emergency':0.8},
                 "Non-ED Admission": {'Elective':0, 'Surgical Emergency':0.4, 'Medical Emergency':0.6}
                 }

los_distributions = {
    'Emergency Department': {'Elective': (1, 0.5), 'Surgical Emergency': (2, 0.7), 'Medical Emergency': (3, 1)},
    'Non-ED Admission': {'Elective': (1.5, 0.6), 'Surgical Emergency': (2.5, 0.8), 'Medical Emergency': (3.5, 1.2)},
    'Elective': {'Elective': (2, 0.7)}
}

arrivals_mapping = {'Emergency Department': {'Monday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                             'Tuesday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                             'Wednesday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                             'Thursday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                             'Friday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                             'Saturday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                             'Sunday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1]},
                    'Non-ED Admission': {'Monday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                         'Tuesday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                         'Wednesday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                         'Thursday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                         'Friday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                         'Saturday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                         'Sunday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1]},
                    'Elective': {'Monday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                 'Tuesday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                 'Wednesday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                 'Thursday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                 'Friday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                 'Saturday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1],
                                 'Sunday': [1,2,4,5,2,5,8,5,6,5,8,5,4,5,2,1,4,5,6,9,8,7,0,1]}}

hospital = BedModel(n_elective_beds=40,
                    n_surgical_emergency_beds=150,
                    n_medical_emergency_beds=450,
                    n_escalation_beds=20, 
                    source_probability=source_prob, 
                    category_probability=category_prob, 
                    los_distributions=los_distributions,
                    time_matrix=arrivals_mapping)

hospital.warm_up_model(warmup_number=warmup_n)


hospital.occupied_medical_emergency_beds


[[754417, 'Emergency Department', 'Medical Emergency', 26],
 [797787, 'Non-ED Admission', 'Medical Emergency', 30],
 [725925, 'Emergency Department', 'Medical Emergency', 4],
 [105483, 'Emergency Department', 'Medical Emergency', 6],
 [181266, 'Emergency Department', 'Medical Emergency', 41],
 [978779, 'Emergency Department', 'Medical Emergency', 89],
 [899025, 'Emergency Department', 'Medical Emergency', 18],
 [164708, 'Emergency Department', 'Medical Emergency', 52],
 [306728, 'Emergency Department', 'Medical Emergency', 8],
 [494918, 'Emergency Department', 'Medical Emergency', 41],
 [108898, 'Emergency Department', 'Medical Emergency', 205],
 [279596, 'Emergency Department', 'Medical Emergency', 54],
 [117102, 'Emergency Department', 'Medical Emergency', 4],
 [969517, 'Emergency Department', 'Medical Emergency', 3],
 [644330, 'Non-ED Admission', 'Medical Emergency', 16],
 [182061, 'Emergency Department', 'Medical Emergency', 9],
 [725866, 'Emergency Department', 'Medical Emergency'